In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.models import load_model
from keras import applications
from keras import optimizers
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
import pickle
from keras import regularizers

import keras.backend as K

from model import Model
K.clear_session()

In [2]:
#Model parameters for target model
batch_size = 128
epochs = 100
lrate = 0.001
decay = 1e-7
num_classes = 10
#initializer in each layer - upto us

In [3]:
X_target = np.load('data/X_target.npy')
y_target = np.load('data/y_target.npy')
y_target_not_one_hot = y_target.copy()
y_target = keras.utils.to_categorical(y_target, num_classes)

X_member, X_nonmember, y_member, y_nonmember = train_test_split(X_target, y_target, test_size=0.5, random_state=42)

In [4]:
target = Model(input_shape=X_target.shape[1:], num_classes=num_classes)

In [6]:
target.train(X_member, y_member, batch_size, epochs, lrate, decay, X_nonmember, y_nonmember)
target.eval()

Training accuracy = 0.996333
Validation accuracy = 0.975333


In [11]:
filepath = 'models/mnist_target.h5'
target.save(filepath)

In [22]:
y_pred_member = target.predict(X_member)
y_pred_non_member = target.predict(X_nonmember)
labels_attack_test = np.vstack((y_member,y_nonmember))
X_attack_test = np.vstack((y_pred_member, y_pred_non_member))
y_attack_test = np.zeros(X_attack_test.shape[0])
y_attack_test[:y_pred_member.shape[0]] = 1

469/469 [==============================] - 2s 3ms/step


In [23]:
x_attack_test_dict = {'X_attack_test':X_attack_test,'y_attack_test':y_attack_test,'labels_attack_test':labels_attack_test}
fname = 'data/attack_test_data'
np.save(fname,x_attack_test_dict)